In [4]:
import numpy as np
import pandas as pd
from google.colab import drive
import ast

In [5]:
prefix = '/content/drive'
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


In [ ]:
hot_100_path = '/content/drive/MyDrive/CIS 450 Project/Hot 100 Audio Features.xlsx'
hot_stuff_path = '/content/drive/MyDrive/CIS 450 Project/Hot Stuff.csv'


In [ ]:
bill_path = '/content/drive/MyDrive/CIS 450 Project/Cleaned Data/BillboardAppearance.csv'
url_path = '/content/drive/MyDrive/CIS 450 Project/Cleaned Data/URL.csv'
song_path = '/content/drive/MyDrive/CIS 450 Project/Cleaned Data/Song.csv'
performer = '/content/drive/MyDrive/CIS 450 Project/Cleaned Data/PerformerTitle.csv'
lyric = '/content/drive/MyDrive/CIS 450 Project/Cleaned Data/Lyric.csv'
haslyric = '/content/drive/MyDrive/CIS 450 Project/Cleaned Data/HasLyric.csv'

In [ ]:
df_l = pd.read_csv(lyric)
df_hl = pd.read_csv(haslyric)

In [ ]:
df_hl.merge(df_l, on='word')

,song_id,word,count,popularity
0,soul deepthe box tops,after,2,312
1,stand by your mantammy wynette,after,1,312
2,the show must go onthree dog night,after,1,312
3,life's been goodjoe walsh,after,1,312
4,i wish it would rainthe temptations,after,1,312
...,...,...,...,...
246492,what about nowdaughtry,nena,5,2957
246493,what about nowdaughtry,pasa,8,1876
246494,what about nowdaughtry,ust,3,4465
246495,what about nowdaughtry,vivo,1,1518


In [ ]:
df_hl.to_csv("HasLyrics.csv", index=False)
files.download("HasLyrics.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_hot_100 = pd.read_excel(hot_100_path)
df_hot_stuff = pd.read_csv(hot_stuff_path)

In [ ]:
# Difference between songs in set2 and set1
set1 = set(df_hot_100['SongID'].tolist())
set2 = set(df_hot_stuff['SongID'].tolist())
# set2 - set1.intersection(set2)
# print(len(df_hot_100['SongID'].tolist()) + len(df_hot_stuff['SongID'].tolist()))

In [ ]:
import requests, json,time
import urllib.parse as parse

audio_features = ['acousticness', 'danceability', 'energy', 'instrumentalness',
       'key', 'liveness', 'loudness', 'mode', 'speechiness','tempo',
       'time_signature', 'valence']

def get_query(song, artist):
    return "https://api.spotify.com/v1/search?query=track%3A{}+artist%3A{}&type=track&offset=0&limit=1".format(parse.quote(song), parse.quote(artist))

def get_audio_query(id):
    return "https://api.spotify.com/v1/audio-features/{}".format(id)

def get_artist_query(artist_id):
    return "https://api.spotify.com/v1/artists/{id}".format(id=artist_id)

def get_audio_features(id):
    default = {feature:'' for feature in audio_features}
    if id=='': return default
    query = get_audio_query(id)
    response = requests.get(query, headers=headers)
    if response.status_code == 200:
        data = json.loads(response.text)
        return {feature:data.get(feature, '') for feature in audio_features}
    else:
        print(response.text)
        return default

def get_genres(author_id):
    if id=='': return ''
    query = get_artist_query(author_id)
    response = requests.get(query, headers=headers)
    if response.status_code == 200:
        data = json.loads(response.text)
        return data.get('genres', '')
    else:
        print(response.text)
        return ''

default = {'spotify_track_album':"", 'spotify_track_duration_ms':"",
       'spotify_track_explicit':"", 'spotify_track_popularity':"", 
       "spotify_genre": "", "spotify_track_id": '', "spotify_track_preview_url":'',
       **{name:'' for name in audio_features}}

def process_response(data):
    item = data['tracks']['items']
    if len(item) == 0:
        return default
    item = item[0]
    album = item.get('album', {}).get('name', '')
    id = item.get("id", '')
    artist_id = item.get('artists', [{}])[0].get('id', '')
    genres = get_genres(artist_id)
    popularity = item.get('popularity', '')
    preview_url = item.get('preview_url', '')
    explicit = 1 if item.get('explicit', False) else 0
    duration = item.get('duration_ms', '')
    audio_features = get_audio_features(id)
    return {'spotify_track_album':album, 'spotify_track_duration_ms':duration,
       'spotify_track_explicit':explicit, 'spotify_track_popularity':popularity, 
       "spotify_genre": genres, "spotify_track_id": id, "spotify_track_preview_url":preview_url, **audio_features}

In [ ]:
hot_stuff_ind = df_hot_stuff.set_index("SongID")

In [ ]:
token='BQDqg3ZiPtnppYDXdskpDBacNRorj4zOuFriCLhdZSH5zQC9ghk7saI3s8cGb7Ib2uXVfeZI--ZANDNYjMbAYbtRuuh7Z-zlZ5VzWG6o9rCwGDMD4_-M-8z1a33ZjTO7MWwooky9H-n6sPac'
headers = {
    'Authorization': 'Bearer {}'.format(token)
}

vals = {name:[] for name in df_hot_100.columns}
for id in (set2-set1):
    time.sleep(2)
    row = hot_stuff_ind.loc[id]
    vals['SongID'].append(id)
    vals['Performer'].append(row['Performer'])
    vals['Song'].append(row['Song'])
    query = get_query(row['Song'], row['Performer'])
    response = requests.get(query, headers=headers)
    if response.status_code == 200:
        data = process_response(json.loads(response.text))
        for key in data:
            vals[key].append(data[key])
    else:
        print(response.text, response.headers.get("retry-after", "no headers"))
        for key in vals:
            if key!="SongID" and key!="Performer" and key!="Song": vals[key].append('')

missing = pd.DataFrame(data=vals)

{
  "error" : {
    "status" : 404,
    "message" : "analysis not found"
  }
}
{
  "error" : {
    "status" : 404,
    "message" : "analysis not found"
  }
}


In [ ]:
df_hot_100 = df_hot_100.append(missing, ignore_index=True)

In [ ]:
df_hot_100.head()

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],3jbT1Y5MoPwEIpZndDDwVq,,Adicto (with Anuel AA & Ozuna),0,270740,91,0.734,0.836,10,-4.803,0,0.0735,0.017,1.64e-05,0.179,0.623,80.002,4
1,The Ones That Didn't Make It Back HomeJustin M...,Justin Moore,The Ones That Didn't Make It Back Home,"['arkansas country', 'contemporary country', '...",,,,,,,,,,,,,,,,,,
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",2VxeLyX666F8uXCJ0dZF8B,,A Star Is Born Soundtrack,0,215733,88,0.572,0.385,7,-6.362,1,0.0308,0.371,0,0.231,0.323,95.799,4
3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,"['dfw rap', 'melodic rap', 'rap']",0Xek5rqai2jcOWCYWJfVCF,,Hollywood's Bleeding,1,196760,86,0.542,0.674,6,-4.169,1,0.21,0.0588,0,0.0955,0.667,76.388,4
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"['dfw rap', 'rap', 'southern hip hop', 'trap']",2biNa12dMbHJrHVFRt8JyO,https://p.scdn.co/mp3-preview/fa6fa6f6f363be29...,Bacc At It Again,1,228185,61,0.948,0.623,8,-5.725,0,0.168,0.00124,1.07e-06,0.0716,0.856,135.979,4


In [ ]:
df_hot_stuff.head()

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
167,http://www.billboard.com/charts/hot-100/1992-0...,5/30/1992,89,Don't Talk Just Kiss,Right Said Fred,don't talk just kissright said fred,1,78,76,7
188,http://www.billboard.com/charts/hot-100/1982-0...,7/24/1982,97,Don't Talk To Strangers,Rick Springfield,don't talk to strangersrick springfield,1,85,2,21
196,http://www.billboard.com/charts/hot-100/1965-1...,11/27/1965,62,Don't Talk To Strangers,The Beau Brummels,don't talk to strangersthe beau brummels,1,52,52,8
210,http://www.billboard.com/charts/hot-100/2001-0...,5/19/2001,98,Don't Talk,Jon B,don't talkjon b,1,97,58,14
212,http://www.billboard.com/charts/hot-100/1982-0...,7/3/1982,81,Don't Talk,Larry Lee,don't talklarry lee,1,82,81,2


To clean the data:
- replace NaN with defaults for spotify_track_id, spotify_track_preview_url, spotify_track_album
- remove duplicate rows (keep the row that comes later)
- check inconsistent names between 2 tables for songs, track albums, genres, performers
- move spotify_genre to seperate table
- clean word popularity by getting rid of filler words
- remember explicit: 1=true, 0=false
- check for inconsistent genres
- change weeks in Billboard to YYYY-MM-DD format


In [ ]:
#check no null rows
print((~df_hot_stuff['SongID'].isna()).all())
print((~df_hot_100['SongID'].isna()).all())

True
True


In [ ]:
# Strip strings
def strip_string(x):
    return x.strip() if isinstance(x, str) else x

df_hot_100 = df_hot_100.apply(strip_string)
df_hot_stuff = df_hot_stuff.apply(strip_string)

In [ ]:
# remove NaNs
df_hot_100 = df_hot_100.fillna("")
df_hot_stuff = df_hot_stuff.fillna("")

In [ ]:
df_hot_100.shape, df_hot_stuff.shape

((29083, 22), (325682, 10))

In [ ]:
#Remove duplicates by spotify_track_id
df_hot_100['SongID'] = df_hot_100['SongID'].apply(lambda x:x.lower().strip())
df_hot_100 = df_hot_100[(df_hot_100['spotify_track_id'].str.len()==0)| (~df_hot_100.duplicated(['spotify_track_id'], keep='last'))]
df_hot_100 = df_hot_100[~df_hot_100.duplicated(['SongID'], keep='last')]

In [ ]:
#Remove duplicates by song id
df_hot_stuff['SongID'] = df_hot_stuff['SongID'].apply(lambda x:x.lower().strip())
df_hot_stuff = df_hot_stuff[~df_hot_stuff.duplicated(['SongID','WeekID'], keep='last')]

In [ ]:
# Check all Song IDs only show up once
grouped = df_hot_100.groupby("SongID")
print((grouped.size() == 1).all())

grouped = df_hot_stuff.groupby(["SongID","WeekID"])
print((grouped.size() == 1).all())

True
True


In [ ]:
values = {"SongID":[], "spotify_genre":[]}
for index, row in df_hot_100.iterrows():
    songid = row['SongID']
    genres = row['spotify_genre'] 
    if len(genres) > 0:
        if isinstance(genres, str): genres = ast.literal_eval(genres)
        for genre in genres:
            values['spotify_genre'].append(genre)
            values['SongID'].append(songid)

genres_df = pd.DataFrame(data=values)

,SongID,spotify_genre
0,"adictotainy, anuel aa & ozuna",pop reggaeton
1,the ones that didn't make it back homejustin m...,arkansas country
2,the ones that didn't make it back homejustin m...,contemporary country
3,the ones that didn't make it back homejustin m...,country
4,the ones that didn't make it back homejustin m...,country road


In [ ]:
genres_df.head()

,SongID,spotify_genre,genre_id
0,"adictotainy, anuel aa & ozuna",pop reggaeton,836
1,the ones that didn't make it back homejustin m...,arkansas country,44
2,the ones that didn't make it back homejustin m...,contemporary country,264
3,the ones that didn't make it back homejustin m...,country,273
4,the ones that didn't make it back homejustin m...,country road,280


In [ ]:
vals = genres_df['spotify_genre'].unique().tolist()
vals.sort()
genremap = {vals[i]:i for i in range(len(vals))}

In [ ]:
genres_df['genre_id'] = genres_df['spotify_genre'].apply(lambda x:genremap.get(x))

In [ ]:
grouped = genres_df.groupby(["SongID", 'genre_id'])
print((grouped.size() == 1).all())

True


In [ ]:
df_hot_100.to_csv("hot_100_processed.csv", index=False)
df_hot_stuff.to_csv("hot_stuff_processed.csv", index=False)
genres_df.to_csv("genres_processed.csv", index=False)

In [ ]:
from google.colab import files
files.download('hot_100_processed.csv') 
files.download("hot_stuff_processed.csv")
files.download("genres_processed.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
Song = df_hot_100[df_hot_100.columns.difference(['Song', 'spotify_genre', 'spotify_track_id', 'spotify_track_preview_url', "Performer"])]
Song.head()

,SongID,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,spotify_track_album,spotify_track_duration_ms,spotify_track_explicit,spotify_track_popularity,tempo,time_signature,valence
0,"adictotainy, anuel aa & ozuna",0.017,0.734,0.836,1.64e-05,10,0.179,-4.803,0,0.0735,Adicto (with Anuel AA & Ozuna),270740,0,91,80.002,4,0.623
1,the ones that didn't make it back homejustin m...,,,,,,,,,,,,,,,,
3,enemiespost malone featuring dababy,0.0588,0.542,0.674,0,6,0.0955,-4.169,1,0.21,Hollywood's Bleeding,196760,1,86,76.388,4,0.667
5,the archertaylor swift,0.12,0.292,0.574,0.00569,0,0.0663,-9.375,1,0.0401,Lover,211240,0,76,124.344,4,0.166
6,vibezdababy,0.113,0.768,0.652,0,1,0.107,-2.708,0,0.307,KIRK,144935,1,88,154.187,4,0.777


In [ ]:
(Song.groupby(['SongID']).size() == 1).all()

True

In [ ]:
Song.to_csv("Song.csv", index=False)
files.download("Song.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
URL = df_hot_100[['spotify_track_preview_url', 'SongID']]
URL = URL[URL['spotify_track_preview_url'].str.len() > 0]

In [ ]:
(URL.groupby("spotify_track_preview_url").size()==1).all()

True

In [ ]:
URL.to_csv("URL.csv", index=False)
files.download("URL.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
Performer = df_hot_100[['Performer', 'SongID', 'Song']]
Performer.head()

,Performer,SongID,Song
0,"Tainy, Anuel AA & Ozuna","adictotainy, anuel aa & ozuna",Adicto
1,Justin Moore,the ones that didn't make it back homejustin m...,The Ones That Didn't Make It Back Home
3,Post Malone Featuring DaBaby,enemiespost malone featuring dababy,Enemies
5,Taylor Swift,the archertaylor swift,The Archer
6,DaBaby,vibezdababy,VIBEZ


In [ ]:
(Performer.groupby(["Performer", 'Song']).size() == 1).all()

True

In [ ]:
Performer.to_csv("PerformerTitle.csv", index=False)
files.download("PerformerTitle.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
Billboard = df_hot_stuff[df_hot_stuff.columns.difference(['Song', 'Performer'])]
Billboard.head()

In [ ]:
(Billboard.groupby(["WeekID", 'Week Position', "SongID"]).size() == 1).all()

True

In [ ]:
Billboard.to_csv("BillboardApperance.csv", index=False)
files.download("BillboardApperance.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ADARSH STUFF

In [46]:
songs_path = '/content/drive/MyDrive/Sophomore/Second Semester/CIS 450 Project/songs1.csv'
word_path = '/content/drive/MyDrive/Sophomore/Second Semester/CIS 450 Project/word_popularity.csv'
h100_path = '/content/drive/MyDrive/Sophomore/Second Semester/CIS 450 Project/Hot 100 Audio Features.xlsx'
hstuff_path = '/content/drive/MyDrive/Sophomore/Second Semester/CIS 450 Project/Hot Stuff.csv'

In [48]:
df_songs = pd.read_csv(songs_path, sep="\t")
df_word_popularity = pd.read_csv(word_path)
df_h100 = pd.read_excel(h100_path)
df_hstuff = pd.read_csv(hstuff_path)

In [49]:
# Remove duplicates from the table that has songs for which we have lyrical information
df_songs_no_dups = df_songs[['msd_artist','msd_title']].drop_duplicates()
df_songs_no_dups

,msd_artist,msd_title
0,Adelitas Way,Scream
1,Western Addiction,A Poor Recipe For Civic Cohesion
2,Casual,I Didn't Mean To
3,The Box Tops,Soul Deep
4,Super Deluxe,Come On (Album Version)
...,...,...
551986,Donald Fagen,Trans-Island Skyway (Album Version)
551987,Rush,Mr. Soul (Album Version)
551988,Reel People,Outta Love (reprise)
551989,Ruper Ordorika,Hamabostean Behin


In [54]:
# Merge Billboard data with lyrical data to determine what songs we have lyrical information for
merged = df_songs_no_dups.merge(df_h100, how="inner", left_on=['msd_artist','msd_title'], right_on= ['Performer','Song'])

In [56]:
# Remove columns that resulted from the merge
songs_with_lyrics = merged.drop(columns=['msd_artist', 'msd_title'])
songs_with_lyrics
songs_with_lyrics.to_csv("songs_with_lyrics.csv", index=False)
files.download("songs_with_lyrics.csv")

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,Soul DeepThe Box Tops,The Box Tops,Soul Deep,"['brill building pop', 'bubblegum pop', 'folk ...",4yPl1mK1oluIrCwI4HInPR,https://p.scdn.co/mp3-preview/9a14555c8355f636...,Dimensions,0.0,147760.0,36.0,0.749,0.530,9.0,-12.940,1.0,0.0326,0.1950,0.000035,0.0748,0.966,121.395,4.0
1,Break My StrideMatthew Wilder,Matthew Wilder,Break My Stride,['new wave pop'],1mCsF9Tw4AkIZOjvZbZZdT,https://p.scdn.co/mp3-preview/06ba33dcbe01b551...,I Don't Speak The Language,0.0,184480.0,69.0,0.917,0.588,10.0,-11.047,1.0,0.0607,0.1950,0.000000,0.0914,0.828,110.077,4.0
2,It's My PartyLesley Gore,Lesley Gore,It's My Party,"['adult standards', 'brill building pop', 'bub...",1Pq47iFLC5U7j8xeNiNcuS,NaN,I'll Cry If I Want To,0.0,140000.0,58.0,0.692,0.494,9.0,-11.662,1.0,0.0635,0.3460,0.000000,0.0446,0.961,128.487,4.0
3,On Bended KneeBoyz II Men,Boyz II Men,On Bended Knee,"['boy band', 'new jack swing', 'pop', 'quiet s...",7MYmo0JJJDmu4MZTSAF9y3,NaN,II,0.0,329800.0,60.0,0.626,0.532,8.0,-8.097,1.0,0.0351,0.4130,0.000000,0.1160,0.197,116.621,4.0
4,Groovin' In The MidnightMaxi Priest,Maxi Priest,Groovin' In The Midnight,"['lovers rock', 'reggae', 'reggae fusion', 'ro...",6BD0vChKASpviIQeCTd7Yy,NaN,Best Of Maxi Priest,0.0,303533.0,31.0,0.652,0.645,10.0,-8.482,0.0,0.0414,0.0401,0.000023,0.0739,0.555,92.945,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5710,Come On Over To My PlaceThe Drifters,The Drifters,Come On Over To My Place,"['adult standards', 'brill building pop', 'bub...",4MRhEMGSsBTaBX8IBNCe1b,https://p.scdn.co/mp3-preview/decf67212240f217...,I'll Take You Where The Music's Playing,0.0,151533.0,39.0,0.716,0.741,1.0,-6.549,0.0,0.0673,0.2970,0.000000,0.0409,0.817,128.099,4.0
5711,She's Not ThereSantana,Santana,She's Not There,"['blues rock', 'classic rock']",44JITu0xanJn3TLMQZJw0G,https://p.scdn.co/mp3-preview/99021e00b5503cfd...,Moonflower,0.0,249026.0,53.0,0.477,0.697,0.0,-7.878,1.0,0.0387,0.4030,0.262000,0.3110,0.832,130.365,4.0
5712,I Missed MeJim Reeves,Jim Reeves,I Missed Me,"['adult standards', 'nashville sound', 'rock-a...",4m3Q6C3o1jYamTikYAm1CI,https://p.scdn.co/mp3-preview/45283b7ad3071738...,The Essential Jim Reeves,0.0,153666.0,21.0,0.474,0.199,7.0,-17.580,1.0,0.0280,0.8110,0.001420,0.1350,0.302,85.011,3.0
5713,Give Me TimeDusty Springfield,Dusty Springfield,Give Me Time,"['adult standards', 'brill building pop', 'bri...",6duLDfk7RheKAup8VFt0P1,NaN,The Look Of Love,0.0,187866.0,11.0,0.433,0.584,4.0,-7.348,0.0,0.0334,0.0468,0.000000,0.2000,0.354,95.876,3.0


In [43]:
all_words = df_word_popularity

In [ ]:
buffer_words = ['so',
 'what',
 'to',
 'do',
 'from',
 'oh',
 'where',
 'a',
 'not',
 'will',
 'an',
 'got',
 'y',
 'yo',
 'he',
 'of',
 'here',
 'en',
 'could',
 'le',
 'have',
 'it',
 'the',
 'too',
 'de',
 'would',
 'se',
 'there',
 'we',
 'how',
 'du',
 'mi',
 'wanna',
 'my',
 'ya',
 'el',
 'more',
 'on',
 'did',
 'are',
 'o',
 'if',
 'be',
 'been',
 'by',
 'un',
 'was',
 'and',
 'then',
 'all',
 'at',
 'is',
 'que',
 'she',
 'for',
 'get',
 'as',
 'te',
 'this',
 'but',
 'si',
 'in',
 'can',
 'la',
 'caus',
 'tu',
 'am',
 'when',
 'were',
 'e',
 'gonna',
 'or',
 'them',
 'es',
 'should',
 'they',
 'come',
 'with',
 'had',
 'that',
 'just']

In [44]:
# Remove more boring buffer words and sort so that we can re-index the popularity values
cleaned_words = all_words[~all_words['word'].isin(buffer_words)]
cleaned_words = cleaned_words.sort_values('popularity', ignore_index=True)
cleaned_words

,word,popularity
0,i,1
1,you,3
2,me,7
3,your,14
4,no,24
...,...,...
4833,santo,4996
4834,pe,4997
4835,gee,4998
4836,colleg,4999


In [45]:
# Re-index by word popularity
for i, tuple in cleaned_words.sort_values('popularity').iterrows():
...     cleaned_words.loc[i, "popularity"] = i+1
cleaned_words
cleaned_words.to_csv("cleaned_words.csv", index=False)
files.download("cleaned_words.csv")

,word,popularity
0,i,1
1,you,2
2,me,3
3,your,4
4,no,5
...,...,...
4833,santo,4834
4834,pe,4835
4835,gee,4836
4836,colleg,4837
